In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import csv
import random
import matplotlib.pyplot as plt
import statistics 
from scipy import stats

plt.rcParams["figure.figsize"] = (6,6)



# THE GRAPH CONTROL PANEL

In [2]:
N = 100       # Number of nodes
setting='home'    #the setting we are evaluating(home,work,school,other)
D=50           #number of days the model runs
threshold=.005    #the threshold of a contact to happen
increase_rate=.02 #the rate of increasing weight per contact
decrease_rate=.02 #the rate of decreasing weight if no contact
base_weight=.1    #the presumed weight for initial graph
lower_weight=.01  #the minimum weight between two node(Max is =1)
threshold_n=1     #threshold of product weight two nodes with same neighbor for increasing weight
increse_rate_n=0  #increse rate weight for two nodes with same neighbor

## 1-Defining Nodes

In [3]:
G = nx.Graph()

In [4]:
#defining a dictionary of nodes
available_groups  = [] #16_age_groups
group_popularity = [] #based on consensus year 95
v_prevalence=dict() #based on meta analysis(data is not exact!! update needed!!)contan(age_group,prevalance,lower CL)
nodes_dic=dict()

with open ('population.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        available_groups.append(row[0])
        group_popularity.append(float(row[1]))
        
with open ('varicella_prevalance.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        v_prevalence[row[0]]=[float(row[1]),float(row[2])]
        
        


for i in range(N):
    while True:
        r=random.random()
        t=random.randint(0,15)
        if r<=group_popularity[t]:
            nodes_dic[f'n{i+1}']={'age':available_groups[t]}
            break


for i in range(N):
    a=v_prevalence[nodes_dic[f'n{i+1}']['age']][0]
    b=v_prevalence[nodes_dic[f'n{i+1}']['age']][1]
    r=np.random.random()
    t=np.random.normal(a,(a-b)) 
    if r<=t:
        nodes_dic[f'n{i+1}']['infection']=1
    else:
        nodes_dic[f'n{i+1}']['infection']=0
        

In [5]:
#importing data for frequency of contact in each setting for comparison
frequency=dict()
with open ('frequency_contact.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        
        frequency[row[0]]=dict()
        for i in range(1,6):
            frequency[row[0]][i]=float(row[i])

### Which `group` each node belongs to?

* We  used `Age specific population for Iran` to define probability of each group

In [6]:

# let's see the number of nodes in each group:

group_count = {}
for n in nodes_dic.keys():
    
    if nodes_dic[n]['age'] in group_count:
        group_count[nodes_dic[n]['age']] = group_count[nodes_dic[n]['age']] + 1
    else:
        group_count[nodes_dic[n]['age']] = 1
group_count

{'20_24': 6,
 '0_4': 10,
 '25_29': 7,
 '35_39': 8,
 '65_69': 4,
 '45_49': 7,
 '30_34': 16,
 '60_64': 3,
 '75+': 5,
 '55_59': 2,
 '15_19': 7,
 '50_54': 5,
 '10_14': 4,
 '40_44': 8,
 '5_9': 6,
 '70_74': 2}

## 2-Defining Connection Rules between Node Groups

### `We define rules in terms of connection probability between each two groups`


In [7]:
#importing rules from .csv to nested dictionary
rules={}  # rules contain ->{location:{contactor:{contactee:contact number}
temp_0={}
temp_1={}
location=[]
contactor=[]
contactee=[]
contact_number=[]


with open ('contact_rules.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        
        location.append(row[0])
        contactor.append(row[1])
        contactee.append(row[2])
        contact_number.append(float(row[3]))


for o in range (0,1280,256):
    for m in range(o,o+256,16):
        for n in range (m,m+16):
            
            temp_0[contactee[n]]=contact_number[n]
        temp_1[contactor[m]]=temp_0.copy()
    rules[location[o]]=temp_1.copy()     

In [8]:

#bulding empty dictinary "contacted" for storing contacts in single rrun
#building empty dictionary for monitoring all contacts during all runs
contacted={}
contacted_all={}
for i in nodes_dic.keys():
    contacted[i]={}
    contacted_all[i]={}
    for j in nodes_dic.keys():
        if i!=j:
            
            contacted[i][j]=0 
            contacted_all[i][j]=[]


## 3- Visualization for `SMALL` graph

### ⚠️ Only for SMALL graphs (i.e. noes < 100)

## 4-Defining Rules to Connecting Nodes and Adjusting `Probabilities/Distances`

* This would be the step that our Naive graph would evolve over the time through a simulation process
* we monitor connection between nodes and update weights based on connections

In [9]:
#in this block we've defiend a function "Run" 
def Run(increase_rate,decrease_rate,threshold,threshold_n,increse_rate_n ,base_weight):
    
    for i in nodes_dic.keys():
        for j in nodes_dic.keys():
            G.add_edge(i,j,weight=base_weight)
            
    
    for d in range (0,D):#we are running the model for 100 days
        #iterating thorough the graph and storing connections in 'contacted'
        
        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                     #age_p is the number of contact between nodes by age
                    age_p=rules[setting][nodes_dic[i]['age']][nodes_dic[j]["age"]]
                    r=random.random()
                    if G[i][j]['weight']*r*age_p > threshold:
                        contacted[i][j]=1
                        contacted_all[i][j].append(1)#storing contacts of each run in this dic
                    else:
                        contacted_all[i][j].append(0)
        #iterating thorough 'contacted' and updating the weights based on it
        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    if contacted[i][j]==1 and G[i][j]['weight']+increase_rate<1 :
                        #we don't want the weight to be 1 or more
                        t=G[i][j]['weight']+increase_rate
                        G.add_edge(i,j,weight=t)

                    elif contacted[i][j]==0 and G[i][j]['weight']-decrease_rate>lower_weight : 
                        #we don't want the weight to be 0 or less
                        t=G[i][j]['weight']-decrease_rate
                        G.add_edge(i,j,weight=t)

        #updating weight between two nodes with same neighbor
        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    for o in nodes_dic.keys():
                        if o!=j and o!=i:

                            if (G[i][o]['weight']*G[i][j]['weight'])>threshold_n and G[o][j]['weight']<1:
                                t=G[o][j]['weight']+increse_rate_n
                                G.add_edge(i,j,weight=t)
    return(contacted_all)


### Visualize the graph after the updates

# 5- Monitroing the graph
### we monitor the graph for age specific number of contact per day and frequeny of contact

In [10]:
    #this functions returns mean absoloute error 'ages specified number of conatacts'

def metric_A(contacted_all):
        L1=[]
        L2=[]
    
        rules_m=dict()
        rules_m[setting]=dict()

        for i in rules[setting].keys():
            rules_m[setting][i]=dict()
            for j in rules[setting][i].keys():
                 rules_m[setting][i][j]=0


        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    t=sum(contacted_all[i][j])/D
                    rules_m[setting][nodes_dic[i]['age']][nodes_dic[j]['age']]+=t
        
        
        #for i in rules_m[setting].keys():
            #for j in rules_m[setting][i].keys():
                
                #print(i,j,'model:',rules_m[setting][i][j],'original:',rules[setting][i][j])

        for i in rules_m[setting].keys():
            for j in rules_m[setting][i].keys():
                L1.append(rules[setting][i][j])
                L2.append(rules_m[setting][i][j])
                
        I=stats.wilcoxon(L1,L2, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
        return(I)

In [11]:
#defining a function that returns mean absoloute error 'frequency of contacts ''
def metric_F(contacted_all):
    n=0
    L1=[]
    L2=[]
    frequency_m=dict()
    for i in frequency.keys():
        frequency_m[i]={1:0,2:0,3:0,4:0,5:0}

    for i in nodes_dic.keys():
        for j in nodes_dic.keys():
            if i!=j:
                t=sum(contacted_all[i][j])/D

                if t==1/D:
                    frequency_m[setting][5]+=1
                    n+=1
                elif t>1/D and t<(1/30) :
                    frequency_m[setting][4]+=1
                    n+=1
                elif t>=(1/30) and t<=(2/30):
                    frequency_m[setting][3]+=1
                    n+=1
                elif t>(2/30) and t <(2/7):
                    frequency_m[setting][2]+=1
                    n+=1
                elif t>=(2/7):
                    n+=1
                    frequency_m[setting][1]+=1

    for i in frequency_m[setting].keys():
        if n!=0:
        
            frequency_m[setting][i]=frequency_m[setting][i]/n
        if n==0:
            
            frequency_m[setting][i]=0
        
            
        
    
    for i in frequency_m[setting].keys():
        
        L1.append(frequency[setting][i])
        L2.append(frequency_m[setting][i])
        
    
    I=stats.wilcoxon(L1,L2, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
    #print('for the first time:',frequency_m[setting][5],'\nless than once a month:',frequency_m[setting][4],'\nabout once or twice a month:',frequency_m[setting][3],'\nabout once or twice a week',frequency_m[setting][2],'\ndaily or almost daily',frequency_m[setting][1])

    return(I)
        


   



# grid-search
### this part of code checks different value sets for function Run 

###### Run(increase_rate,decrease_rate,threshold,threshold_n,increse_rate_n ,base_weight)

In [16]:
for i in [.01,.02,.03]:
    
    for j in [.01,.02,.03]:
        
        for k in [.008,.005,.002]:
            
            
                print ("Run")
                
                R=Run(i,j,k,threshold_n,increse_rate_n,base_weight)
                
        
                print(i,j,k)
                print('Metric','%f' % (metric_A(R)[1]))
      
        

Run
0.01 0.01 0.008 1024
Metric 0.000000
Run
0.01 0.01 0.005 1024
Metric 0.000000
Run
0.01 0.01 0.002 1024
Metric 0.000000
Run
0.01 0.02 0.008 1024
Metric 0.000000
Run


KeyboardInterrupt: 